In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# Global variable for the WebDriver instance
driver = None

def open_browser():
    """
    Initialize and open the browser to the Copilot chat page.
    """
    global driver
    # Set up the Edge WebDriver
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    
    # Navigate to the Copilot chat page
    driver.get("https://m365.cloud.microsoft/chat")
    
    # Pause to allow the page to load
    time.sleep(5)
    print("Browser opened and navigated to the Copilot chat page.")

def type_and_submit_text(text):
    """
    Switch to the iframe, type text into the chat box, and submit by pressing Enter.
    
    Args:
        text (str): The text to input into the Copilot chat box.
    """
    global driver
    try:
        # Switch to the iframe by XPath
        iframe = driver.find_element(By.XPATH, '//*[@id="iframe:d870f6cd-4aa5-4d42-9626-ab690c041429"]')
        driver.switch_to.frame(iframe)
        print("Switched to iframe.")

        # Locate the text box inside the iframe
        text_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="m365-chat-editor-target-element"]'))
        )
        print("Text box found.")

        # Type the desired text into the text box
        text_box.send_keys(text)
        print(f'Text "{text}" typed into the text box.')

        # Press Enter to submit the input
        text_box.send_keys(Keys.ENTER)
        print("Text submitted (Enter key pressed).")

        # Switch back to the main content (optional)
        driver.switch_to.default_content()
    except Exception as e:
        print(f"An error occurred while typing and submitting text: {e}")

def wait_for_response():
    """
    Wait until Copilot has finished generating the response.
    """
    global driver
    try:
        # Switch to the iframe where the stop generation button is located
        iframe_xpath = '//*[@id="iframe:d870f6cd-4aa5-4d42-9626-ab690c041429"]'
        WebDriverWait(driver, 15).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, iframe_xpath)))
        print("Switched to iframe for waiting for response.")

        stop_button_xpath = '/html/body/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[3]/div/div[1]/div[2]/div/div[2]/div[2]/button/span[2]'

        # Wait until the stop generation button is present
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, stop_button_xpath))
        )
        print("Stop generation button is present. Waiting for response to finish.")

        # Now wait until the stop generation button is no longer visible
        WebDriverWait(driver, 300).until(
            EC.invisibility_of_element_located((By.XPATH, stop_button_xpath))
        )
        print("Response generation finished.")

        # Switch back to the main content (optional)
        driver.switch_to.default_content()
    except Exception as e:
        print(f"An error occurred while waiting for the response: {e}")

def get_copilot_response():
    """
    Retrieve Copilot's full response by executing JavaScript to access the innerText of the response container.
    """
    global driver
    try:
        # Switch to the iframe where the response is located
        iframe_xpath = '//*[@id="iframe:d870f6cd-4aa5-4d42-9626-ab690c041429"]'
        WebDriverWait(driver, 15).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, iframe_xpath)))
        print("Switched to iframe for response retrieval.")

        # Wait for the specific p[1] element to be present
        p1_xpath = '/html/body/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[3]//p[1]'
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, p1_xpath))
        )
        print("p[1] element is present.")

        # Locate the parent container of the response
        response_container_xpath = '/html/body/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[3]'
        response_container = driver.find_element(By.XPATH, response_container_xpath)

        # Use JavaScript to get the innerText of the response container
        response_text = driver.execute_script("return arguments[0].innerText;", response_container)

        print(f"Copilot's full response:\n{response_text}")

        # Switch back to the main content (optional)
        driver.switch_to.default_content()
        return response_text
    except Exception as e:
        print(f"An error occurred while retrieving Copilot's response: {e}")
        return None

def create_new_chat():
    """
    Switch to the appropriate iframe and click the 'New Chat' button to start a new conversation.
    """
    global driver
    try:
        # Switch to the same iframe as before (assuming it's in the same iframe)
        iframe = driver.find_element(By.XPATH, '//*[@id="iframe:d870f6cd-4aa5-4d42-9626-ab690c041429"]')
        driver.switch_to.frame(iframe)
        print("Switched to iframe for New Chat button.")

        # Locate the 'New Chat' button and click it
        new_chat_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div/div/div/div/div[1]/div[1]/div[2]/div[3]/div/button'))
        )
        new_chat_button.click()
        print("New Chat button clicked.")

        # Switch back to the main content (optional)
        driver.switch_to.default_content()
    except Exception as e:
        print(f"An error occurred while clicking the New Chat button: {e}")

def close_browser():
    """
    Close the browser instance and clean up.
    """
    global driver
    if driver:
        driver.quit()
        print("Browser closed.")

# Main Execution
if __name__ == "__main__":
    try:
        open_browser()
        type_and_submit_text("Write a 100 line poem")  # Enter and submit first message
        wait_for_response()  # Wait for the response to finish
        get_copilot_response()  # Retrieve and print the response
        create_new_chat()  # Start a new chat
        type_and_submit_text("This is the second chat!")  # Enter and submit second message
        wait_for_response()  # Wait for the response to finish
        get_copilot_response()  # Retrieve and print the response
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        pass  # close_browser()
